# IN3063 - Coursework

### Team members:
1. Aymen 
2. Philip
3. Adam
4. Taha

## Libraries

In [ ]:
import math
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt

## Sigmoid & ReLU

- By Aymen

In [2]:
# sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# backpropagation
# ...

## Softmax

- By Aymen

## Dropout

- By Adam

In [ ]:
def dropout():
    pass # placeholder